In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
pd.set_option('display.max_rows', None)
from plotly.subplots import make_subplots
import seaborn as sns
import datetime

#Plotly Libraris
import plotly.express as px

# Minmax scaler
from sklearn.preprocessing import MinMaxScaler

#itertools
import itertools

#dataframe display settings
pd.set_option('display.max_columns', 5000000)
pd.set_option('display.max_rows', 50000000)

#to suppress un-necessary warnings
import warnings  
warnings.filterwarnings('ignore')


#Package to flatten python lists
from pandas.core.common import flatten

In [ ]:
covid_data = pd.read_csv('covid_19_data.csv')
covid_confirmed = pd.read_csv('time_series_covid_19_confirmed.csv')
covid_deaths = pd.read_csv('time_series_covid_19_deaths.csv')
covid_recovered = pd.read_csv('time_series_covid_19_recovered.csv')

In [ ]:
covid_confirmed.describe()

In [ ]:
NAN = [(c, covid_data[c].isna().mean()*100) for c in covid_data]
NAN = pd.DataFrame(NAN, columns=["column_name", "percentage"])
NAN

<font color='black'> 34 % of Province/State are unknown we will fill nan values with Unknown. </font>0

In [ ]:
covid_data["Province/State"]= covid_data["Province/State"].fillna('Unknown')

Change Data Type for "Confirmed","Deaths" and "Recovered"  columns to int

In [ ]:
covid_data[["Confirmed","Deaths","Recovered"]] =covid_data[["Confirmed","Deaths","Recovered"]].astype(int)

Replacing "Mainland China" with "China" 

In [ ]:
covid_data['Country/Region'] = covid_data['Country/Region'].replace('Mainland China', 'China')

* Creating new feature "Active_case" 
* Active_case = Confirmed - Deaths - Recovered

In [ ]:
covid_data['Active_case'] = covid_data['Confirmed'] - covid_data['Deaths'] - covid_data['Recovered']

In [ ]:
covid_data.head()

In [ ]:
Data = covid_data[covid_data['ObservationDate'] == max(covid_data['ObservationDate'])].reset_index()

### Time series stats 

In [ ]:
base_stats = pd.DataFrame(columns=['Dates','Confirmed','Deaths','Recovered','Active'])
base_stats['Dates'] = covid_confirmed.columns[4:]

base_stats['Confirmed'] = base_stats['Dates'].apply(lambda x: covid_confirmed[x].sum())
base_stats['Deaths'] = base_stats['Dates'].apply(lambda x: covid_deaths[x].sum())
base_stats['Recovered'] = base_stats['Dates'].apply(lambda x: covid_recovered[x].sum())
base_stats.reset_index(drop=False, inplace=True)
base_stats['Active'] = base_stats['index'].apply(lambda x: (base_stats['Confirmed'][x]-(base_stats['Deaths'][x]+base_stats['Recovered'][x])))
base_stats.head()

In [ ]:
latest_stats_fig = go.Figure()
latest_stats_fig.add_trace(go.Treemap(labels = ['Confirmed','Active','Recovered','Deaths'],
                                     parents = ['','Confirmed','Confirmed','Confirmed'],
                                     values = [base_stats['Confirmed'].sum(), base_stats['Active'].sum(), base_stats['Recovered'].sum(), base_stats['Deaths'].sum()],
                                      branchvalues="total", marker_colors = ['#118ab2','#ef476f','#06d6a0','#073b4c'],
                                      textinfo = "label+text+value",
                                      outsidetextfont = {"size": 30, "color": "darkblue"},
                                      marker = {"line": {"width": 2}},
                                        pathbar = {"visible": False}
                                     ))
latest_stats_fig.update_layout(#width=1000, 
                               height=300)
latest_stats_fig.show()

### Cases over time around the world

In [ ]:
base_stats_fig = go.Figure()

for column in base_stats.columns.to_list()[2:6]:
    color_dict = {
      "Confirmed": "#118ab2",
      "Active": "#ef476f",
      "Recovered": "#06d6a0",
      "Deaths": "#073b4c"
        }
    base_stats_fig.add_trace(
        go.Scatter(
            x = base_stats['Dates'],
            y = base_stats[column],
            name = column,
            line = dict(color=color_dict[column]),
            hovertemplate ='<br><b>Date</b>: %{x}'+'<br><i>Count</i>:'+'%{y}',
        )
    )
    
for column in base_stats.columns.to_list()[2:6]:
    color_dict = {
      "Confirmed": "#149ECC",
      "Active": "#F47C98",
      "Recovered": "#24F9C1",
      "Deaths": "#0C6583"
        }
    base_stats_fig.add_trace(
        go.Scatter(
            x = base_stats['Dates'],
            y = base_stats['index'].apply(lambda x: (base_stats[column][x-7:x].sum())/7 if x>7 else (base_stats[column][0:x].sum())/7),
            name = column+" 7-day Moving Avg.",
            line = dict(dash="dash", color=color_dict[column]), showlegend=False,
            hovertemplate = '<br><b>Date</b>: %{x}'+'<br><i>7-day moving avg.</i>: %{y}'
        )
    )
    
base_stats_fig.update_layout(
    updatemenus=[
        dict(
        buttons=list(
            [dict(label = 'All Cases',
                  method = 'update',
                  args = [{'visible': [True, True, True, True, True, True, True, True]},
                          {'title': 'All Cases',
                           'showlegend':True}]),
             dict(label = 'Confirmed',
                  method = 'update',
                  args = [{'visible': [True, False, False, False, True, False, False, False]},
                          {'title': 'Confirmed',
                           'showlegend':True}]),
             dict(label = 'Active',
                  method = 'update',
                  args = [{'visible': [False, False, False, True, False, False, False, True]},
                          {'title': 'Active',
                           'showlegend':True}]),
             dict(label = 'Recovered',
                  method = 'update',
                  args = [{'visible': [False, False, True, False, False, False, True, False]},
                          {'title': 'Recovered',
                           'showlegend':True}]),
             dict(label = 'Deaths',
                  method = 'update',
                  args = [{'visible': [False, True, False, False, False, True, False, False]},
                          {'title': 'Deaths',
                           'showlegend':True}]),
            ]),
             type = "dropdown",
             direction="down",
#             pad={"r": 10, "t": 40},
             showactive=True,
             x=0,
             xanchor="left",
             y=1.25,
             yanchor="top"
        ),
        dict(
        buttons=list(
            [dict(label = 'Linear Scale',
                  method = 'relayout',
                  args = [{'yaxis': {'type': 'linear'}},
                          {'title': 'All Cases',
                           'showlegend':True}]),
             dict(label = 'Log Scale',
                  method = 'relayout',
                  args = [{'yaxis': {'type': 'log'}},
                          {'title': 'Confirmed',
                           'showlegend':True}]),
            ]),
             type = "dropdown",
             direction="down",
#             pad={"r": 10, "t": 10},
             showactive=True,
             x=0,
             xanchor="left",
             y=1.36,
             yanchor="top"
        )
    ])

base_stats_fig.update_xaxes(showticklabels=False)
base_stats_fig.update_layout(
    #height=600, width=600, 
    title_text="Basic Statistics for Covid19", title_x=0.5, title_font_size=20,
                            legend=dict(orientation='h',yanchor='top',y=1.15,xanchor='right',x=1), paper_bgcolor="mintcream",
                            xaxis_title="Date", yaxis_title="Number of Cases")
base_stats_fig.show()

In [ ]:
covid_data.head()

### Increase in cases

In [ ]:
daily_case_fig = make_subplots(rows=2, cols=2, vertical_spacing=0.05, horizontal_spacing=0.04, # shared_yaxes=True,
                           subplot_titles=('Confirmed','Active','Recovered','Deaths'),
                            x_title='Dates', y_title='# of Cases',)

daily_case_fig.add_trace(go.Bar(x=base_stats['Dates'], y=base_stats['index'].apply(lambda x: base_stats['Confirmed'][x]-base_stats['Confirmed'][x-1:x].sum()),
                              name='Confirmed',hovertemplate = '<br><b>Date</b>: %{x}'+'<br><i>Confirmed Count</i>: %{y}',
                                marker=dict(color='#118ab2')),row=1, col=1)
daily_case_fig.add_trace(go.Scatter(x=base_stats['Dates'], y=base_stats['index'].apply(lambda x: (base_stats['Confirmed'][x-7:x].sum()-base_stats['Confirmed'][x-8:x-1].sum())/7 if x>0 else 0),
                             name='7-day moving average', hovertemplate = '<br><b>Date</b>: %{x}'+'<br><i>7-day average</i>: %{y}', showlegend=False,
                                    line=dict(dash="dash", color='#149ECC')),row=1, col=1)

daily_case_fig.add_trace(go.Bar(x=base_stats['Dates'], y=base_stats['index'].apply(lambda x: base_stats['Active'][x]-base_stats['Active'][x-1:x].sum()), 
                             name='Active',hovertemplate = '<br><b>Date</b>: %{x}'+'<br><i>Active Count</i>: %{y}',
                               marker=dict(color='#ef476f')),row=1, col=2)
daily_case_fig.add_trace(go.Scatter(x=base_stats['Dates'], y=base_stats['index'].apply(lambda x: (base_stats['Active'][x-7:x].sum()-base_stats['Active'][x-8:x-1].sum())/7 if x>0 else 0),
                             name='7-day moving average', hovertemplate = '<br><b>Date</b>: %{x}'+'<br><i>7-day average</i>: %{y}', showlegend=False,
                                    line=dict(dash="dash", color='#F47C98')),row=1, col=2)

daily_case_fig.add_trace(go.Bar(x=base_stats['Dates'], y=base_stats['index'].apply(lambda x: base_stats['Recovered'][x]-base_stats['Recovered'][x-1:x].sum()), 
                              name='Recovered',hovertemplate = '<br><b>Date</b>: %{x}'+'<br><i>Recovered Count</i>: %{y}',
                               marker=dict(color='#06d6a0')),row=2, col=1)
daily_case_fig.add_trace(go.Scatter(x=base_stats['Dates'], y=base_stats['index'].apply(lambda x: (base_stats['Recovered'][x-7:x].sum()-base_stats['Recovered'][x-8:x-1].sum())/7 if x>0 else 0),
                             name='7-day moving average', hovertemplate = '<br><b>Date</b>: %{x}'+'<br><i>7-day average</i>: %{y}', showlegend=False,
                                    line=dict(dash="dash", color='#24F9C1')),row=2, col=1)

daily_case_fig.add_trace(go.Bar(x=base_stats['Dates'], y=base_stats['index'].apply(lambda x: base_stats['Deaths'][x]-base_stats['Deaths'][x-1:x].sum()), 
                              name='Deaths',hovertemplate = '<br><b>Date</b>: %{x}'+'<br><i>Death Count</i>: %{y}',
                               marker=dict(color='#073b4c')),row=2, col=2)
daily_case_fig.add_trace(go.Scatter(x=base_stats['Dates'], y=base_stats['index'].apply(lambda x: (base_stats['Deaths'][x-7:x].sum()-base_stats['Deaths'][x-8:x-1].sum())/7 if x>0 else 0),
                             name='7-day moving average', hovertemplate = '<br><b>Date</b>: %{x}'+'<br><i>7-day average</i>: %{y}', line=dict(dash="dash", color='#0C6583')),row=2, col=2)




daily_case_fig.update_xaxes(showticklabels=False)
daily_case_fig.update_layout(
    #height=600, width=1100, 
    title_text="Daily change in cases of Covid19", title_x=0.5, title_font_size=20,
                            legend=dict(orientation='h',yanchor='top',y=1.1,xanchor='right',x=1), paper_bgcolor="mintcream")


daily_case_fig.show() 

 **Confirmed cases in each Country**

In [ ]:
Data_per_country = covid_data.groupby(["Country/Region"])["Confirmed","Active_case","Recovered","Deaths"].sum().reset_index().sort_values("Confirmed",ascending=False).reset_index(drop=True)


In [ ]:
headerColor = 'grey'
rowEvenColor = 'lightgrey'
rowOddColor = 'white'

fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Country</b>','<b>Confirmed Cases</b>'],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
      
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      Data_per_country['Country/Region'],
      Data_per_country['Confirmed'],
      ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*len(Data_per_country)],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])
fig.update_layout(
    title='Confirmed Cases In Each Country',
)
fig.show()

#### Evolution of coronavirus over time.

In [ ]:
data_per_country = covid_data.groupby(["Country/Region","ObservationDate"])[["Confirmed","Active_case","Recovered","Deaths"]].sum().reset_index().sort_values("ObservationDate",ascending=True).reset_index(drop=True)


In [ ]:
fig = px.choropleth(data_per_country, locations=data_per_country['Country/Region'],
                    color=data_per_country['Active_case'],locationmode='country names', 
                    hover_name=data_per_country['Country/Region'], 
                    color_continuous_scale=px.colors.sequential.Tealgrn,
                    animation_frame="ObservationDate")
fig.update_layout(

    title='Evolution of active cases In Each Country',
    template='plotly_dark'
)
fig.show()

In [ ]:
fig = px.choropleth(data_per_country, locations=data_per_country['Country/Region'],
                    color=data_per_country['Recovered'],locationmode='country names', 
                    hover_name=data_per_country['Country/Region'], 
                    color_continuous_scale=px.colors.sequential.deep,
                    animation_frame="ObservationDate")
fig.update_layout(
    title='Evolution of recovered cases In Each Country',
)
fig.show()

In [ ]:
fig = px.choropleth(data_per_country, locations=data_per_country['Country/Region'],
                    color=data_per_country['Deaths'],locationmode='country names', 
                    hover_name=data_per_country['Country/Region'], 
                    color_continuous_scale=px.colors.sequential.Tealgrn,
                    animation_frame="ObservationDate")
fig.update_layout(
    title='Evolution of deaths In Each Country',
    template='plotly_dark'
)
fig.show()

In [ ]:
fig = go.Figure(data=[go.Bar(
            x=Data_per_country['Country/Region'][0:10], y=Data_per_country['Confirmed'][0:10],
            text=Data_per_country['Confirmed'][0:10],
            textposition='auto',
            marker_color='pink',
            

        )])
fig.update_layout(
    title='Most 10 infected Countries',
    xaxis_title="Countries",
    yaxis_title="Confirmed Cases",
        template='plotly_white'

)
fig.show()

#### Recorvered cases in each Country 

In [ ]:
Recovered_per_country = covid_data.groupby(["Country/Region"])["Recovered"].sum().reset_index().sort_values("Recovered",ascending=False).reset_index(drop=True)

In [ ]:
headerColor = 'grey'
rowEvenColor = 'lightgrey'
rowOddColor = 'white'

fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Country</b>','<b>Recovered Cases</b>'],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      Recovered_per_country['Country/Region'],
      Recovered_per_country['Recovered'],
      ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*len(Data_per_country)],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])
fig.update_layout(
    title='Recovered Cases In Each Country',
)
fig.show()


In [ ]:
fig = px.pie(Recovered_per_country, values=Recovered_per_country['Recovered'], names=Recovered_per_country['Country/Region'],
             title='Recovered cases',
            )
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(
    template='plotly_white'
)
fig.show()

In [ ]:
fig = go.Figure(data=[go.Bar(
            x=Recovered_per_country['Country/Region'][0:10], y=Recovered_per_country['Recovered'][0:10],
            text=Recovered_per_country['Recovered'][0:10],
            textposition='auto',
            marker_color='lavender',

        )])
fig.update_layout(
    title='Most 10 infected Countries',
    xaxis_title="Countries",
    yaxis_title="Recovered Cases",
    template='plotly_white'
)
fig.show()

#### Active cases in each Country

In [ ]:
Active_per_country = covid_data.groupby(["Country/Region"])["Active_case"].sum().reset_index().sort_values("Active_case",ascending=False).reset_index(drop=True)


In [ ]:
headerColor = 'grey'
rowEvenColor = 'lightgrey'
rowOddColor = 'white'

fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Country</b>','<b>Active Cases</b>'],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      Active_per_country['Country/Region'],
      Active_per_country['Active_case'],
      ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*len(Data_per_country)],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])
fig.update_layout(
    title='Active Cases In Each Country',
)
fig.show()

In [ ]:
fig = go.Figure(data=[go.Bar(
            x=Active_per_country['Country/Region'][0:10], y=Active_per_country['Active_case'][0:10],
            text=Active_per_country['Active_case'][0:10],
           
        )])
fig.update_layout(
    title='Most 10 infected Countries',
    xaxis_title="Countries",
    yaxis_title="Active Cases",
    template='plotly_white'
)
fig.show()


#### Deaths cases in each Country

In [ ]:
Deaths_per_country = covid_data.groupby(["Country/Region"])["Deaths"].sum().reset_index().sort_values("Deaths",ascending=False).reset_index(drop=True)

In [ ]:
headerColor = 'grey'
rowEvenColor = 'lightgrey'
rowOddColor = 'white'

fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Country</b>','<b>Deaths</b>'],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      Deaths_per_country['Country/Region'],
      Deaths_per_country['Deaths'],
      ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*len(Data_per_country)],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])
fig.update_layout(
    title='Deaths In Each Country',
)
fig.show()

In [ ]:
fig = go.Figure(data=[go.Bar(
            x=Deaths_per_country['Country/Region'][0:10], y=Deaths_per_country['Deaths'][0:10],
            text=Deaths_per_country['Deaths'][0:10],
            textposition='auto',
            marker_color=' green'

        )])
fig.update_layout(
    title='Most 10 infected Countries',
    xaxis_title="Countries",
    yaxis_title="Deaths",
        template='plotly_white'

)
fig.show()


### Predictions

In [ ]:
base_stats_inc_df = pd.DataFrame(columns=['Index', 'Dates', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'Daily Inc.'])
base_stats_inc_df[['Index', 'Dates', 'Confirmed', 'Deaths', 'Recovered', 'Active']] = base_stats[['index', 'Dates', 'Confirmed', 'Deaths', 'Recovered', 'Active']]
base_stats_inc_df['Daily Inc.'] = base_stats['index'].apply(lambda x: base_stats['Confirmed'][x]-base_stats['Confirmed'][x-1:x].sum())

In [ ]:
days = np.array(base_stats_inc_df[['Index']]).reshape(-1, 1)
days_ex = []
for i in range(len(days)+30):
    days_ex = days_ex+[[i]]

In [ ]:
prediction_df = pd.DataFrame(columns=['Index', 'Confirmed Pred', 'Deaths Pred', 'Recovered Pred', 'Active Pred', 'Daily Inc. Pred'])
prediction_df['Index'] = list(flatten(days_ex))

In [ ]:
for col in base_stats_inc_df.columns[2:]:

    count = np.array(base_stats_inc_df[[col]]).reshape(-1, 1)

    X_train_confirmed, X_test_confirmed, y_train_confirmed, y_test_confirmed = train_test_split(
                                                        days[50:], count[50:], 
                                                        test_size=0.05, shuffle=False)

    MAE, RSE, R2 = [], [], []
    for j in range(1,10):
        #creating the model
        poly = PolynomialFeatures(degree=j)
        train_x_poly = poly.fit_transform(X_train_confirmed)
        
        regr_poly = linear_model.LinearRegression()
        regr_poly.fit(train_x_poly, y_train_confirmed)
        
        y_pred_poly = regr_poly.predict(poly.fit_transform(X_test_confirmed))
        MAE.append(np.mean(np.absolute(y_pred_poly - y_test_confirmed)))
        RSE.append(np.mean((y_pred_poly - list(flatten(y_test_confirmed))) ** 2))
        R2.append(r2_score(y_pred_poly, list(flatten(y_test_confirmed))))
        
    deg = RSE.index(min(RSE))+1
    #print("best deg for column {} is {}".format(col, deg))

    poly = PolynomialFeatures(degree=deg)
    train_x_poly = poly.fit_transform(X_train_confirmed)

    regr_poly = linear_model.LinearRegression()
    regr_poly.fit(train_x_poly, y_train_confirmed)
    col_name = col+' Pred'
    prediction_df[col_name] = list(flatten(regr_poly.predict(poly.fit_transform(days_ex))))

In [ ]:
prediction_fig = go.Figure()
pred_dict = {
  "Confirmed": ["#118ab2", 'Confirmed', 'Predicted Confirmed','#149ECC'],
  "Active": ["#ef476f", 'Deaths', 'Predicted Deaths', '#F47C98'],
  "Recovered": ["#06d6a0", 'Recovered', 'Predicted Recovered','#24F9C1'],
  "Deaths": ["#073b4c", 'Active', 'Predicted Active','#0C6583'],
  "Daily Inc.": ["black", 'Daily Inc.', 'Predicted Daily Inc.','grey']
    }

for z in base_stats_inc_df.columns[2:]:
    
    z_pred = z+' Pred'
    prediction_fig.add_trace(go.Scatter(x=list(flatten(days)), y=base_stats_inc_df[z],
                                       line=dict(color=pred_dict[z][0]), name = pred_dict[z][1],
                                       hovertemplate ='<br><b>Day number</b>: %{x}'+'<br><i>No.of cases </i>:'+'%{y}'))
    
    prediction_fig.add_trace(go.Scatter(x=list(flatten(days_ex))[50:], y=prediction_df[z_pred][50:],
                                       line=dict(dash="dash", color='black'), visible=False, name = pred_dict[z][2],
                                       hovertemplate ='<br><b>Day number</b>: %{x}'+'<br><i>Predicted no.of cases </i>:'+'%{y}'))

    
    
    
prediction_fig.update_layout(
    updatemenus=[
        dict(
        buttons=list(
            [dict(label = 'Confirmed',
                  method = 'update',
                  args = [{'visible': [True, True, False, False, False, False, False, False, False, False]},
                          {'title': 'Confirmed Cases',
                           'showlegend':True}]),
             dict(label = 'Deaths',
                  method = 'update',
                  args = [{'visible': [False, False, True, True, False, False, False, False, False, False]},
                          {'title': 'Deaths Cases',
                           'showlegend':True}]),
             dict(label = 'Recovered',
                  method = 'update',
                  args = [{'visible': [False, False, False, False, True, True, False, False, False, False]},
                          {'title': 'Recovered Cases',
                           'showlegend':True}]),
             dict(label = 'Active',
                  method = 'update',
                  args = [{'visible': [False, False, False, False, False, False, True, True, False, False]},
                          {'title': 'Active Cases',
                           'showlegend':True}]),
             dict(label = 'Daily Inc.',
                  method = 'update',
                  args = [{'visible': [False, False, False, False, False, False, False, False, True, True]},
                          {'title': 'Daily Inc. Cases',
                           'showlegend':True}]),
            ]),
             type = "buttons",
             direction="down",
#             pad={"r": 10, "t": 40},
             showactive=True,
#              x=1.01,
#              xanchor="left",
             y=1.1,
             yanchor="top"
        )
    ])


prediction_fig.update_xaxes(showticklabels=False)
prediction_fig.update_layout(
    #height=500, width=1100, 
    title_text="Prediction for Covid19 Cases", title_x=0.5, title_font_size=20,
                            legend=dict(orientation='h',yanchor='top',y=1.12,xanchor='right',x=1), paper_bgcolor="mintcream",
                            xaxis_title="Number of Days <br> (Click on the buttons at the left to see the predictions)", yaxis_title="Count")
prediction_fig.show()

# <span style='color:navy;'> Coronavirus in my country India </span>

In [ ]:
Data_india = covid_data [(covid_data['Country/Region'] == 'India') ].reset_index(drop=True)
Data_india.head()

In [ ]:
fig = go.Figure(go.Bar(
            x=Data_india['ObservationDate'],
            y=Data_india['Active_case'],
    marker_color='rgb(13,48,100)'
           ))
fig.update_layout(
    title='Active cases In Each Day',
    template='plotly_white',
     xaxis_title="Active cases",
    yaxis_title="Days",
)
fig.show()

In [ ]:
fig = go.Figure(go.Bar(
            x=Data_india['ObservationDate'],
            y=Data_india['Active_case'],
    marker_color='rgb(13,48,100)'
           ))
fig.update_layout(
    title='Active cases In Each Day',
    template='plotly_white',
     xaxis_title="Active cases",
    yaxis_title="Days",
)
fig.show()

In [ ]:
fig = go.Figure(go.Bar(
            x=Data_india['ObservationDate'],
            y=Data_india['Recovered'],
    marker_color='rgb(13,48,100)'
           ))
fig.update_layout(
    title='Recovered cases In Each Day',
    template='plotly_white',
     xaxis_title="Recovered cases",
    yaxis_title="Days",
)
fig.show()

In [ ]:
fig = go.Figure(go.Bar(
            x=Data_india['ObservationDate'],
            y=Data_india['Deaths'],
    marker_color='rgb(13,48,100)'
           ))
fig.update_layout(
    title='Deaths In Each Day',
    template='plotly_white',
     xaxis_title="Deaths",
    yaxis_title="Days",
)
fig.show()

#### Get Last Update

In [ ]:
Data_india_last = Data_india[Data_india['ObservationDate'] == max(Data_india['ObservationDate'])].reset_index()
Data_india_last

In [ ]:
colors = ['rgb(2,58,88)','rgb(65,171,93)', 'rgb(127,0,0)']
labels = ["Active cases","Recovered","Deaths"]
values = Data_india_last.loc[0, ["Active_case","Recovered","Deaths"]]

fig = go.Figure(data=[go.Pie(labels=labels,
                             values=values)])
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
fig.show()

<a id='chi'></a>
#  <font color='red'> **Coronavirus in China** </font>


In [ ]:
Data_China = covid_data [(covid_data['Country/Region'] == 'China') ].reset_index(drop=True)
Data_China.head()

#### Get last update in china

In [ ]:
Data_china_last = Data_China[Data_China['ObservationDate'] == max(Data_China['ObservationDate'])].reset_index()
Data_china_last.head()

#### Confirmed cases in every Province/State in china

In [ ]:
Data_china_per_state= Data_china_last.groupby(["Province/State"])["Confirmed","Active_case","Recovered","Deaths"].sum().reset_index().sort_values("Confirmed",ascending=False).reset_index(drop=True)


In [ ]:
fig = px.pie(Data_china_per_state, values=Data_china_per_state['Confirmed'], names=Data_china_per_state['Province/State'],
             title='Confirmed cases in China',
            hole=.2)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
fig = go.Figure(go.Bar(
            x=Data_china_per_state['Active_case'],
            y=Data_china_per_state['Province/State'],
            orientation='h',
            marker_color='#DC3912',))
fig.update_layout(
    title='Active Cases In Each Province/State',
    template='plotly_white',
    xaxis_title="Active Cases",
    yaxis_title="Province/State",
)
fig.show()

In [ ]:
fig = go.Figure(go.Bar(
            x=Data_china_per_state['Recovered'],
            y=Data_china_per_state['Province/State'],
            orientation='h',
            marker_color='green',))
fig.update_layout(
    title='Active Cases In Each Province/State',
    template='plotly_white',
    xaxis_title="Recovered Cases",
    yaxis_title="Province/State",
)
fig.show()

In [ ]:
fig = go.Figure(go.Bar(
            x=Data_china_per_state['Deaths'],
            y=Data_china_per_state['Province/State'],
            orientation='h',
            marker_color='black',))
fig.update_layout(
    title='Deaths In Each Province/State',
    template='plotly_white',
    xaxis_title="Deaths",
    yaxis_title="Province/State",
)
fig.show()

#### Get total cases in China

In [ ]:
Data_china_total= Data_china_last.groupby(["Country/Region"])["Confirmed","Deaths","Recovered","Active_case"].sum().reset_index().reset_index(drop=True)


In [ ]:
labels = ["Active cases","Recovered","Deaths"]
values = Data_china_total.loc[0, ["Active_case","Recovered","Deaths"]]
df = px.data.tips()
fig = px.pie(Data_china_total, values=values, names=labels, color_discrete_sequence=['green','royalblue','darkblue'], hole=0.5)
fig.update_layout(
    title='Total cases in China : '+str(Data_china_total["Confirmed"][0]),
)
fig.show()

<a id='us'></a>
#  <font color='blue'> **Coronavirus in United States** </font>


In [ ]:
Data_US = covid_data [(covid_data['Country/Region'] == 'US') ].reset_index(drop=True)


#### Get last update in US

In [ ]:
Data_us_last = Data_US[Data_US['ObservationDate'] == max(Data_US['ObservationDate'])].reset_index()

In [ ]:
Data_us_total= Data_us_last.groupby(["Country/Region"])["Confirmed","Deaths","Recovered","Active_case"].sum().reset_index().reset_index(drop=True)

In [ ]:
labels = ["Active cases","Recovered","Deaths"]
values = Data_us_total.loc[0, ["Active_case","Recovered","Deaths"]]
df = px.data.tips()
fig = px.pie(Data_us_total, values=values, names=labels, color_discrete_sequence=['royalblue','darkblue','green'], hole=0.5)
fig.update_layout(
    title='Total cases in United States : '+str(Data_us_total["Confirmed"][0]),
)
fig.show()

#### Cases in every Province/State in US

In [ ]:
Data_us_per_state= Data_us_last.groupby(["Province/State"])["Confirmed","Active_case","Deaths"].sum().reset_index().sort_values("Confirmed",ascending=False).reset_index(drop=True)

In [ ]:
fig = px.pie(Data_us_per_state, values=Data_us_per_state['Confirmed'], names=Data_us_per_state['Province/State'],
             title='Confirmed cases in United States',
            hole=.2)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
fig = px.pie(Data_us_per_state, values=Data_us_per_state['Active_case'], names=Data_us_per_state['Province/State'],
             title='Active cases in United States',
            hole=.2)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
fig = px.pie(Data_us_per_state, values=Data_us_per_state['Deaths'], names=Data_us_per_state['Province/State'],
             title='Deaths in United States',
            hole=.2)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

<a id='spa'></a>
#  <font color='purple'> **Coronavirus in Spain** </font>

In [ ]:
Data_Spain = covid_data [(covid_data['Country/Region'] == 'Spain') ].reset_index(drop=True)


In [ ]:
Data_spain = Data_Spain[Data_Spain['ObservationDate'] == max(Data_Spain['ObservationDate'])].reset_index()
Data_spain_last= Data_spain.groupby(["Country/Region"])["Confirmed","Deaths","Recovered","Active_case"].sum().reset_index().reset_index(drop=True)


In [ ]:
labels = ["Active cases","Recovered","Deaths"]
values = Data_spain_last.loc[0, ["Active_case","Recovered","Deaths"]]
df = px.data.tips()
fig = px.pie(Data_spain_last, values=values, names=labels, color_discrete_sequence=['royalblue','green','darkblue'], hole=0.5)
fig.update_layout(
    title='Total cases in Spain : '+str(Data_spain_last["Confirmed"][0]),
)
fig.show()

In [ ]:
Data_spain_per_state= Data_spain.groupby(["Province/State"])["Confirmed","Deaths","Recovered","Active_case"].sum().reset_index().sort_values("Confirmed",ascending=False).reset_index(drop=True)

In [ ]:

fig = px.treemap(Data_spain_per_state, path=['Province/State'], values=Data_spain_per_state['Confirmed'], height=700,
                 title='Confirmed cases in Spain', color_discrete_sequence = px.colors.qualitative.Dark2)
fig.data[0].textinfo = 'label+text+value'
fig.show()

In [ ]:
fig = px.treemap(Data_spain_per_state, path=['Province/State'], values=Data_spain_per_state['Recovered'], height=700,
                 title='Recovered cases in Spain', color_discrete_sequence = px.colors.qualitative.Dark2)
fig.data[0].textinfo = 'label+text+value'
fig.show()

In [ ]:

fig = px.treemap(Data_spain_per_state, path=['Province/State'], values=Data_spain_per_state['Active_case'], height=700,
                 title='Active cases in Spain', color_discrete_sequence = px.colors.sequential.deep)
fig.data[0].textinfo = 'label+text+value'
fig.show()

In [ ]:
fig = px.treemap(Data_spain_per_state, path=['Province/State'], values=Data_spain_per_state['Deaths'], height=700,
                 title='Deaths in Spain', color_discrete_sequence = px.colors.sequential.deep)
fig.data[0].textinfo = 'label+text+value'
fig.show()

<a id='usw'></a>

#  <font color='lylac'> **US X The rest of the word** </font>


In [ ]:
Data_US_op= Data_US.groupby(["ObservationDate","Country/Region"])["Confirmed","Deaths","Recovered","Active_case"].sum().reset_index().reset_index(drop=True)

In [ ]:
Data_Word = covid_data [(covid_data['Country/Region'] != 'US') ].reset_index(drop=True)
Data_WORD_last = Data_Word[Data_Word['ObservationDate'] == max(Data_Word['ObservationDate'])].reset_index()
Data_us_total= Data_us_last.groupby(["Country/Region"])["Confirmed","Deaths","Recovered","Active_case"].sum().reset_index().reset_index(drop=True)
Data_word_total= Data_WORD_last.groupby(["ObservationDate"])["Confirmed","Deaths","Recovered","Active_case"].sum().reset_index().reset_index(drop=True)

Data_Word_op= Data_Word.groupby(["ObservationDate"])["Confirmed","Deaths","Recovered","Active_case"].sum().reset_index().reset_index(drop=True)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=Data_US_op.index, y=Data_US_op['Confirmed'],
                    mode='lines',
                    name='Confirmed cases in US'))


fig.add_trace(go.Scatter(x=Data_Word_op.index, y=Data_Word_op['Confirmed'],
                    mode='lines',
                    name='Confirmed cases in The Rest Of The Word'))

fig.update_layout(
    title='Evolution of Confirmed cases over time in US and The Rest Of The Word',
        template='plotly_white'

)

fig.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=Data_US_op.index, y=Data_US_op['Active_case'],
                    mode='lines',
                    name='Active cases in US'))


fig.add_trace(go.Scatter(x=Data_Word_op.index, y=Data_Word_op['Active_case'],
                    mode='lines',
                    name='Active cases in The Rest Of The Word'))

fig.update_layout(
    title='Evolution of Active cases over time in US and The Rest Of The Word',
        template='plotly_dark'

)

fig.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=Data_US_op.index, y=Data_US_op['Recovered'],
                    mode='lines',
                    name='Recovered cases in US'))


fig.add_trace(go.Scatter(x=Data_Word_op.index, y=Data_Word_op['Recovered'],
                    mode='lines',
                    name='Recovered cases in The Rest Of The Word'))

fig.update_layout(
    title='Evolution of Recovered cases over time in US and The Rest Of The Word',
        template='plotly_dark'

)

fig.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=Data_US_op.index, y=Data_US_op['Deaths'],
                    mode='lines',
                    name='Deaths in US'))


fig.add_trace(go.Scatter(x=Data_Word_op.index, y=Data_Word_op['Deaths'],
                    mode='lines',
                    name='Deathsin The Rest Of The Word'))
fig.update_layout(
    title='Evolution of Deaths over time in US and The Rest Of The Word',
        template='plotly_white'

)

fig.show()